# Classical decomposition model

# Load libraries

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Load data

In [2]:
df = sns.load_dataset("flights")

In [3]:
df.head()

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121


In [4]:
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str))
df.set_index('date', inplace=True)
df = df[['passengers']]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_28312\1354901155.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str))


In [5]:
df.head()

,passengers
date,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


# Exploratory Data Analysis

In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['passengers'],
    mode='lines+markers',
    marker=dict(size=5),
    line=dict(width=2),
    name='Passengers'
))

fig.update_layout(
    title='Number of Passengers Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    template='plotly_white',
    hovermode='x'
)

fig.show()

# ⚠️ Split your data set into train/test ⚠️

- A train/test split is crucial in time series analysis to assess a model's ability to generalize and accurately predict future values, ensuring it performs well on unseen data rather than just memorizing historical patterns.

- In time series analysis, you can’t use a standard random train/test split because it would break the temporal order of the data. Since time series data is sequential, each observation depends on previous points; shuffling the data would remove this critical relationship and lead to unrealistic, unreliable results when evaluating model performance.

- Splitting techniques:
  - Split the data chronologically, using the earlier portion for training and the later portion for testing.
  - Use an expanding window where each training set includes all data up to the prediction point, gradually increasing as you move forward in time.

  `from sklearn.model_selection import TimeSeriesSplit`
  
  `tscv = TimeSeriesSplit(n_splits=5)`


In [7]:
# Split the data chronologically
train_size = int(len(df) * 0.8)  # 80% for training
train, test = df[:train_size], df[train_size:]

# 📈 Classical decomposition model

## 1. Linear Regression Model
From the linear regression context, we know that a model takes the form:

$$ Y_i = \beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \dots + \beta_p X_{ip} + \epsilon_i
 $$

 where:
 - $Y_i$ is the response variable
 - $\beta_j$'s are the parameters (coefficients)
 - $X_{ij}$'s are the predictor variables
 - $\epsilon_i$ is the error term.

### 2. Time Series Decomposition Model

Using this idea, the following model accounts for trend and seasonality in a time series dataset:

$$ Y_t = m_t + s_t + \epsilon_t
 $$

 where:
 - $Y_t$ is the response variable
 - $m_t$ is the trend component
 - $s_t$ is the seasonal component
 - $\epsilon_t$ is the error term (noise, rnadom fluctuations)

### 3. Modeling the Trend Component

We can typically model the trend component with low order polynomials:

$$ m_t = \sum^p_{i=0}\beta_it^i$$

where $p$ represents the order of the polynomial, $\beta_i$ are coefficients and $t^i$ representes powers of time $t$.

### 4. Modeling the Seasonal Component

We can account for seasonality of period $s$ by defining a categorial variable with $s$ levels and model this with $s-1$ "dummy" variables:

$$ s_t = \sum^{s-1}_{j=1}\gamma_j D_{jt}$$

where $D_{jt}$ are the dummy variables for each season (e.g., month), and $\gamma_j$ are coefficients. The summation goes up to $s-1$ because one level is usually omitted to avoid multicollinearity.

### 5. Complete model

Combining the trend, seasonal, and error components, the full model is:

$$Y_t = \sum^p_{i=0}\beta_it^i + \sum^{s-1}_{j=1}\gamma_j D_{jt} + \epsilon_t$$

or in other words:

$$Y_t = \beta_0 + \beta_1 t + \beta_2 t^2 + \gamma_1 D_{1t} + \gamma_2 D_{2t} + \dots + \gamma_{s-1} D_{(s-1)t} + \epsilon_t
$$

# 🎨 Let’s get our hands dirty with Python!

For this example, we are going to work with the dataset `AirPassengers` that contains the monthly totals of international airline passengers between 1949 and 1960.

In [8]:
# our data
train.head()

,passengers
date,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


Now, we need to prepare our data to use the model described above.

### Creating a time variable $t$

In time series analysis, adding a time variable $t$ allows us to explicitly model **trends** in the data. By representing time as a numerical variable, we enable models to capture any linear or non-linear changes over time (e.g., an upward or downward trend). This step is especially useful in regression-based time series models or when decomposing a time series into trend, seasonal, and residual components.

In [9]:
# Create the time variable `t` in the train DataFrame
train['t'] = range(1, len(train) + 1)

train.head()

,passengers,t
date,,
1949-01-01,112,1
1949-02-01,118,2
1949-03-01,132,3
1949-04-01,129,4
1949-05-01,121,5


### Introducing month as a seasonal variable

In time series data with known seasonality, such as monthly data, certain patterns may repeat at regular intervals. For the Air Passengers dataset, we might expect higher passenger numbers in certain months (e.g., during summer or holiday seasons) and lower numbers in others. Introducing "month" as a categorical variable allows us to capture this **seasonal** pattern.

In [10]:
# Extract month from the date index and convert it to a categorical variable
train['month'] = train.index.month.astype(str)  # Converting to string to treat as a categorical variable

train.head()

,passengers,t,month
date,,,
1949-01-01,112,1,1
1949-02-01,118,2,2
1949-03-01,132,3,3
1949-04-01,129,4,4
1949-05-01,121,5,5


### Data pre-processing

In the graph, you might notice that the **variation in passenger numbers** is not consistent over time. When this happens—meaning that the "spread" or "volatility" of data points increases or decreases over time—we call it **heteroskedasticity**.

To make the model handle this better, we can apply a **log transformation** to the response variable (passenger counts). The log transformation reduces the impact of large numbers and can help stabilize the variance, making the model more effective.


In [11]:
# Apply a log transformation to the passengers column
train['log_passengers'] = np.log(train['passengers'])

train.head()

,passengers,t,month,log_passengers
date,,,,
1949-01-01,112,1,1,4.718499
1949-02-01,118,2,2,4.770685
1949-03-01,132,3,3,4.882802
1949-04-01,129,4,4,4.859812
1949-05-01,121,5,5,4.795791


### Modeling the trend
We're using linear regression to capture the trend in the data, but we're doing it on the log-transformed passenger counts.

- We transformed the passengers data with log to stabilize the variance, making it easier to see the underlying trend.

- We use $t$ (time) as the predictor for the trend, and add a constant term to allow the model to have an intercept.

In [12]:
# Prepare the data
X = train[['t']]  # Use `t` as the predictor
y = train['log_passengers']

# Fit a linear regression model with log-transformed response
model = LinearRegression()
model.fit(X, y)

# Get fitted values (predicted values) from the model
train['fitted_trend'] = model.predict(X)

In [13]:
fig = go.Figure()

# Add the log-transformed passenger data as a line plot
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['log_passengers'],
    mode='lines+markers',
    name='Log(Passengers)',
    customdata=np.stack((train.index.strftime('%Y-%m'), train['passengers']), axis=-1),
    hovertemplate='<b>Date</b>: %{customdata[0]}<br>' +
                  '<b>Log(Passengers)</b>: %{y:.2f}<br>' +
                  '<b>Actual Passengers</b>: %{customdata[1]}'
))

# Add the fitted model trend line as a red line
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['fitted_trend'],
    mode='lines',
    name='Fitted Model',
    line=dict(color='red')
))

# Customize layout
fig.update_layout(
    title='Trend',
    xaxis_title='Date',
    yaxis_title='Log of Passengers',
    template='plotly_white'
)

fig.show()

We can express the **trend** $m_t$ as:

$$ \log(\text{Passengers}) = \beta_0 + \beta_1 \cdot t + \epsilon $$


### Model seasonality

Now we're modeling seasonality in the AirPassengers dataset by using month as a categorical predictor.

In [14]:
# Convert 'month' to categorical (one-hot encode)
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)
month_dummies = one_hot_encoder.fit_transform(train[['month']])

X = month_dummies  # Only using 'month' for seasonality modeling
y = train['log_passengers']

# Fit the linear regression model to capture seasonality
model = LinearRegression()
model.fit(X, y)

# Get the fitted seasonal values (predictions)
train['seasonal_fitted'] = model.predict(X)

In [15]:
# Create the Plotly figure
fig = go.Figure()

# Add the log-transformed passenger data
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['log_passengers'],
    mode='lines+markers',
    name='Log(Passengers)',
    customdata=np.stack((train.index.strftime('%Y-%m'), train['passengers']), axis=-1),
    hovertemplate='<b>Date</b>: %{customdata[0]}<br>' +
                  '<b>Log(Passengers)</b>: %{y:.2f}<br>' +
                  '<b>Actual Passengers</b>: %{customdata[1]}'
))

# Add the fitted seasonal model
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['seasonal_fitted'],
    mode='lines',
    name='Fitted Seasonal Model',
    line=dict(color='red')
))

# Customize layout
fig.update_layout(
    title='Seasonality',
    xaxis_title='Date',
    yaxis_title='Log of Passengers',
    template='plotly_white'
)

fig.show()

We can express the **seasonality** $s_t$ as:

$$ \log(\text{Passengers}) = \text{Intercept} + \beta_{\text{Feb}} \cdot \text{Feb} + \beta_{\text{Mar}} \cdot \text{Mar} + \beta_{\text{Apr}} \cdot \text{Apr} + \dots + \beta_{\text{Dec}} \cdot \text{Dec} + \epsilon $$


### If we add trend and seasonality...

$$ Y_t = m_t + s_t + \epsilon_t
 $$

 $\rightarrow$

$$
\log(\text{Passengers}) = \beta_0 + \beta_1 \cdot t + \beta_{\text{Feb}} \cdot \text{Feb} + \beta_{\text{Mar}} \cdot \text{Mar} + \beta_{\text{Apr}} \cdot \text{Apr} + \dots + \beta_{\text{Dec}} \cdot \text{Dec} + \epsilon
$$


In [16]:
# Combine trend and seasonality features
X = np.column_stack((train['t'], month_dummies))  # Stack 't' and month dummies as features
y = train['log_passengers']

# Fit the linear regression model with both trend and seasonality
model = LinearRegression()
model.fit(X, y)

# Get fitted values (predicted values) from the model
train['fitted'] = model.predict(X)

In [17]:
fig = go.Figure()

# Add the log-transformed passenger data
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['log_passengers'],
    mode='lines+markers',
    name='Log(Passengers)',
    customdata=np.stack((train.index.strftime('%Y-%m'), train['passengers']), axis=-1),
    hovertemplate='<b>Date</b>: %{customdata[0]}<br>' +
                  '<b>Log(Passengers)</b>: %{y:.2f}<br>' +
                  '<b>Actual Passengers</b>: %{customdata[1]}'
))

# Add the fitted trend + seasonality model
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['fitted'],
    mode='lines',
    name='Fitted Model',
    line=dict(color='red')
))

# Customize layout
fig.update_layout(
    title='First model: Trend + Seasonality',
    xaxis_title='Date',
    yaxis_title='Log of Passengers',
    template='plotly_white'
)

fig.show()

#Red is prediction Blue is the actual value

See how much better our model already fits our data just by accounting for trend and seasonality?

But remember… we need to “undo” our log transformation, and we can do this by exponentiating the fitted values.

In [18]:
# Undo the log transformation by exponentiating the fitted values
train['fitted_original_scale'] = np.exp(train['fitted'])

In [19]:
fig = go.Figure()

# Add the original passenger data
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['passengers'],
    mode='lines+markers',
    name='Actual Passengers',
    customdata=np.stack((train.index.strftime('%Y-%m'), train['passengers']), axis=-1),
    hovertemplate='<b>Date</b>: %{customdata[0]}<br>' +
                  '<b>Actual Passengers</b>: %{customdata[1]}'
))

# Add the fitted model on the original scale
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['fitted_original_scale'],
    mode='lines',
    name='Fitted Model (Original Scale)',
    line=dict(color='red')
))

# Customize layout
fig.update_layout(
    title='First model: Trend + Seasonality (Original Scale)',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    template='plotly_white'
)

# Show the figure
fig.show()

# Performance metrics

### Performance on train dataset

We want to make sure our model is at least learning from our train dataset. We want to make sure our model has low bias.

In [20]:
# Train dataset
mse = mean_squared_error(train['passengers'], train['fitted_original_scale'])
rmse = np.sqrt(mse)

In [21]:
rmse

12.57157137859886

In [22]:
train.passengers.describe()

count    115.000000
mean     239.947826
std       91.347788
min      104.000000
25%      170.000000
50%      229.000000
75%      306.000000
max      491.000000
Name: passengers, dtype: float64

An RMSE of 12.57 implies that the model’s predictions are, on average, about 12.57 passengers away from the actual counts, which is a small error relative to the mean and the variability in the data.

### Performance on test dataset

We want to make sure our model is not a memorization machine and that is capable of of predicting in data it has not seen. We want to make sure our model is not overfitting.

In [23]:
# Prepare trend and seasonality for the test set
test['t'] = range(len(train) + 1, len(train) + len(test) + 1)
test['month'] = test.index.month.astype(str)
test['log_passengers'] = np.log(test['passengers'])

# One-hot encode the 'month' column for seasonality in the test set
month_dummies_test = one_hot_encoder.transform(test[['month']])  # Use transform, not fit_transform

# Combine trend and seasonality features for test set
X_test = np.column_stack((test['t'], month_dummies_test))

# Make predictions on the test set
test['fitted_log'] = model.predict(X_test)
test['fitted_original_scale'] = np.exp(test['fitted_log'])  # Exponentiate to original scale

# Compute RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(test['passengers'], test['fitted_original_scale']))
print(f"Test RMSE: {test_rmse:.2f}")

Test RMSE: 52.84


In [24]:
test.passengers.describe()

count     29.000000
mean     440.310345
std       79.514196
min      310.000000
25%      391.000000
50%      419.000000
75%      472.000000
max      622.000000
Name: passengers, dtype: float64

- An RMSE of 52.84 indicates that, on average, the model's predictions on the test set are about 52.84 passengers away from the actual values.

- An RMSE of 52.84 could indicate that the model is reasonably accurate but has room for improvement.

- The Train RMSE is significantly lower than 52.84, which could suggest some degree of overfitting. However, given the heteroskedasticity in the data, this difference may also reflect the model's difficulty in predicting periods of high variance, which are more challenging to forecast accurately.

- Given the heteroskedasticity, we might consider using **Mean Absolute Percentage Error (MAPE)** as an additional metric, as it provides a relative error percentage. This could give a clearer picture of performance, especially in periods with high variability.



In [25]:
fig = go.Figure()

# Add actual passenger data from df
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['passengers'],
    mode='lines+markers',
    name='Actual Passengers'
))

# Add fitted values for the training set
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['fitted_original_scale'],
    mode='lines',
    name='Fitted Model (Train)',
    line=dict(color='red')
))

# Add fitted values for the test set
fig.add_trace(go.Scatter(
    x=test.index,
    y=test['fitted_original_scale'],
    mode='lines',
    name='Fitted Model (Test)',
    line=dict(color='orange')
))

# Customize layout
fig.update_layout(
    title='Air Passengers: Actual vs. Fitted Model (Train and Test)',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    template='plotly_white'
)

# Show the figure
fig.show()

# Forecasting

In [26]:
# Step 1: Generate future dates for the next five years (60 months)
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=60, freq='MS')

# Step 2: Create a new DataFrame for these future dates
future_df = pd.DataFrame(index=future_dates)
future_df['t'] = range(len(train) + len(test) + 1, len(train) + len(test) + 61)  # Continuing t from the last train/test value
future_df['month'] = future_df.index.month.astype(str)  # Extract month as a categorical variable

# Step 3: One-hot encode the 'month' column for seasonality
month_dummies_future = one_hot_encoder.transform(future_df[['month']])  # Use the same encoder as before

# Step 4: Combine trend and seasonality features
X_future = np.column_stack((future_df['t'], month_dummies_future))

# Step 5: Make predictions using the fitted model
future_df['predicted_log'] = model.predict(X_future)
future_df['predicted_passengers'] = np.exp(future_df['predicted_log'])  # Convert log predictions back to original scale

In [27]:
fig = go.Figure()

# Plot actual data
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['passengers'],
    mode='lines+markers',
    name='Actual Passengers',
))

# Plot fitted model for training data
fig.add_trace(go.Scatter(
    x=train.index,
    y=train['fitted_original_scale'],
    mode='lines',
    name='Fitted Model (Train)',
    line=dict(color='red')
))

# Plot fitted model for test data
fig.add_trace(go.Scatter(
    x=test.index,
    y=test['fitted_original_scale'],
    mode='lines',
    name='Fitted Model (Test)',
    line=dict(color='orange')
))

# Plot future predictions
fig.add_trace(go.Scatter(
    x=future_df.index,
    y=future_df['predicted_passengers'],
    mode='lines',
    name='Forecasted Passengers (Next 5 Years)',
    line=dict(color='green') #, dash='dash')
))

# Customize layout
fig.update_layout(
    title='Air Passengers: Historical Data and 5-Year Forecast',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    template='plotly_white'
)

fig.show()